<a href="https://colab.research.google.com/github/mxnuag/Movie-Recommendar/blob/main/ML_movie.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Content-based Movie Recommendation System

In [1]:
import pandas as pd
import numpy as np

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
movies = pd.read_csv('/content/tmdb_5000_movies.csv')
credits = pd.read_csv('/content/tmdb_5000_credits.csv')

In [6]:
movies.shape

(4803, 20)

In [7]:
credits.shape

(1830, 4)

In [5]:
# merging both datasets
movies = movies.merge(credits, on='title')
movies.shape

(4809, 23)

In [6]:
movies.head(1)

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,...,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,movie_id,cast,crew
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...",...,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800,19995,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."


In [7]:
movies.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4809 entries, 0 to 4808
Data columns (total 23 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   budget                4809 non-null   int64  
 1   genres                4809 non-null   object 
 2   homepage              1713 non-null   object 
 3   id                    4809 non-null   int64  
 4   keywords              4809 non-null   object 
 5   original_language     4809 non-null   object 
 6   original_title        4809 non-null   object 
 7   overview              4806 non-null   object 
 8   popularity            4809 non-null   float64
 9   production_companies  4809 non-null   object 
 10  production_countries  4809 non-null   object 
 11  release_date          4808 non-null   object 
 12  revenue               4809 non-null   int64  
 13  runtime               4807 non-null   float64
 14  spoken_languages      4809 non-null   object 
 15  status               

In [11]:
# relevant columns:
# genres
# id
# keywords
# title
# overview
# cast
# crew
movies = movies[['movie_id', 'title', 'overview', 'genres', 'keywords', 'cast', 'crew']]
movies.head(1)

,movie_id,title,overview,genres,keywords,cast,crew
0,19995,Avatar,"In the 22nd century, a paraplegic Marine is di...","[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...","[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...","[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."


In [12]:
movies.columns

Index(['movie_id', 'title', 'overview', 'genres', 'keywords', 'cast', 'crew'], dtype='object')

In [13]:
# check for missing data
movies.isnull().sum()

movie_id    0
title       0
overview    0
genres      0
keywords    0
cast        0
crew        1
dtype: int64

In [14]:
movies.dropna(inplace=True)

In [15]:
movies.shape

(1831, 7)

In [16]:
movies.isnull().sum()

movie_id    0
title       0
overview    0
genres      0
keywords    0
cast        0
crew        0
dtype: int64

In [17]:
# check for duplicate data
movies.duplicated().sum()

0

In [18]:
movies.iloc[0].genres

'[{"id": 28, "name": "Action"}, {"id": 12, "name": "Adventure"}, {"id": 14, "name": "Fantasy"}, {"id": 878, "name": "Science Fiction"}]'

In [19]:
movies.iloc[0].keywords

'[{"id": 1463, "name": "culture clash"}, {"id": 2964, "name": "future"}, {"id": 3386, "name": "space war"}, {"id": 3388, "name": "space colony"}, {"id": 3679, "name": "society"}, {"id": 3801, "name": "space travel"}, {"id": 9685, "name": "futuristic"}, {"id": 9840, "name": "romance"}, {"id": 9882, "name": "space"}, {"id": 9951, "name": "alien"}, {"id": 10148, "name": "tribe"}, {"id": 10158, "name": "alien planet"}, {"id": 10987, "name": "cgi"}, {"id": 11399, "name": "marine"}, {"id": 13065, "name": "soldier"}, {"id": 14643, "name": "battle"}, {"id": 14720, "name": "love affair"}, {"id": 165431, "name": "anti war"}, {"id": 193554, "name": "power relations"}, {"id": 206690, "name": "mind and soul"}, {"id": 209714, "name": "3d"}]'

In [20]:
# convert genres and keywords into a list

import ast
def convert(obj):
    list = [i['name'] for i in ast.literal_eval(obj)]
    return list


In [21]:
movies['genres'] = movies['genres'].apply(convert)

In [22]:
movies['keywords'] = movies['keywords'].apply(convert)

In [23]:
movies.head(2)

,movie_id,title,overview,genres,keywords,cast,crew
0,19995,Avatar,"In the 22nd century, a paraplegic Marine is di...","[Action, Adventure, Fantasy, Science Fiction]","[culture clash, future, space war, space colon...","[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."
1,285,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...","[Adventure, Fantasy, Action]","[ocean, drug abuse, exotic island, east india ...","[{""cast_id"": 4, ""character"": ""Captain Jack Spa...","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de..."


In [24]:
movies.iloc[0].cast

'[{"cast_id": 242, "character": "Jake Sully", "credit_id": "5602a8a7c3a3685532001c9a", "gender": 2, "id": 65731, "name": "Sam Worthington", "order": 0}, {"cast_id": 3, "character": "Neytiri", "credit_id": "52fe48009251416c750ac9cb", "gender": 1, "id": 8691, "name": "Zoe Saldana", "order": 1}, {"cast_id": 25, "character": "Dr. Grace Augustine", "credit_id": "52fe48009251416c750aca39", "gender": 1, "id": 10205, "name": "Sigourney Weaver", "order": 2}, {"cast_id": 4, "character": "Col. Quaritch", "credit_id": "52fe48009251416c750ac9cf", "gender": 2, "id": 32747, "name": "Stephen Lang", "order": 3}, {"cast_id": 5, "character": "Trudy Chacon", "credit_id": "52fe48009251416c750ac9d3", "gender": 1, "id": 17647, "name": "Michelle Rodriguez", "order": 4}, {"cast_id": 8, "character": "Selfridge", "credit_id": "52fe48009251416c750ac9e1", "gender": 2, "id": 1771, "name": "Giovanni Ribisi", "order": 5}, {"cast_id": 7, "character": "Norm Spellman", "credit_id": "52fe48009251416c750ac9dd", "gender": 

In [25]:
# take only top 3 actors from cast
def convert3(obj):
    obj = ast.literal_eval(obj)[:3]
    list = [i['name'] for i in obj]
    return list

In [26]:
movies['cast'] = movies['cast'].apply(convert3)

In [27]:
movies.head(2)

,movie_id,title,overview,genres,keywords,cast,crew
0,19995,Avatar,"In the 22nd century, a paraplegic Marine is di...","[Action, Adventure, Fantasy, Science Fiction]","[culture clash, future, space war, space colon...","[Sam Worthington, Zoe Saldana, Sigourney Weaver]","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."
1,285,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...","[Adventure, Fantasy, Action]","[ocean, drug abuse, exotic island, east india ...","[Johnny Depp, Orlando Bloom, Keira Knightley]","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de..."


In [28]:
movies.iloc[0].crew

'[{"credit_id": "52fe48009251416c750aca23", "department": "Editing", "gender": 0, "id": 1721, "job": "Editor", "name": "Stephen E. Rivkin"}, {"credit_id": "539c47ecc3a36810e3001f87", "department": "Art", "gender": 2, "id": 496, "job": "Production Design", "name": "Rick Carter"}, {"credit_id": "54491c89c3a3680fb4001cf7", "department": "Sound", "gender": 0, "id": 900, "job": "Sound Designer", "name": "Christopher Boyes"}, {"credit_id": "54491cb70e0a267480001bd0", "department": "Sound", "gender": 0, "id": 900, "job": "Supervising Sound Editor", "name": "Christopher Boyes"}, {"credit_id": "539c4a4cc3a36810c9002101", "department": "Production", "gender": 1, "id": 1262, "job": "Casting", "name": "Mali Finn"}, {"credit_id": "5544ee3b925141499f0008fc", "department": "Sound", "gender": 2, "id": 1729, "job": "Original Music Composer", "name": "James Horner"}, {"credit_id": "52fe48009251416c750ac9c3", "department": "Directing", "gender": 2, "id": 2710, "job": "Director", "name": "James Cameron"},

In [29]:
def fetch_director(obj):
    obj = ast.literal_eval(obj)
    director = [i['name'] for i in obj if i['job']=='Director']
    return director

In [30]:
movies['crew'] = movies['crew'].apply(fetch_director)

In [31]:
movies.rename(columns={'crew':'director'}, inplace=True)

In [32]:
movies.head(2)

,movie_id,title,overview,genres,keywords,cast,director
0,19995,Avatar,"In the 22nd century, a paraplegic Marine is di...","[Action, Adventure, Fantasy, Science Fiction]","[culture clash, future, space war, space colon...","[Sam Worthington, Zoe Saldana, Sigourney Weaver]",[James Cameron]
1,285,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...","[Adventure, Fantasy, Action]","[ocean, drug abuse, exotic island, east india ...","[Johnny Depp, Orlando Bloom, Keira Knightley]",[Gore Verbinski]


In [33]:
# convert overview from string to list
movies['overview'] = movies['overview'].apply(lambda x:x.split())

In [34]:
movies.head(2)

,movie_id,title,overview,genres,keywords,cast,director
0,19995,Avatar,"[In, the, 22nd, century,, a, paraplegic, Marin...","[Action, Adventure, Fantasy, Science Fiction]","[culture clash, future, space war, space colon...","[Sam Worthington, Zoe Saldana, Sigourney Weaver]",[James Cameron]
1,285,Pirates of the Caribbean: At World's End,"[Captain, Barbossa,, long, believed, to, be, d...","[Adventure, Fantasy, Action]","[ocean, drug abuse, exotic island, east india ...","[Johnny Depp, Orlando Bloom, Keira Knightley]",[Gore Verbinski]


In [36]:
# remove spaces from names in genres, keywords, cast, director
movies['genres'] = movies['genres'].apply(lambda x:[i.replace(" ","") for i in x])
movies['keywords'] = movies['keywords'].apply(lambda x:[i.replace(" ","") for i in x])
movies['cast'] = movies['cast'].apply(lambda x:[i.replace(" ","") for i in x])
movies['director'] = movies['director'].apply(lambda x:[i.replace(" ","") for i in x])
movies.head(2)

,movie_id,title,overview,genres,keywords,cast,director
0,19995,Avatar,"[In, the, 22nd, century,, a, paraplegic, Marin...","[Action, Adventure, Fantasy, ScienceFiction]","[cultureclash, future, spacewar, spacecolony, ...","[SamWorthington, ZoeSaldana, SigourneyWeaver]",[JamesCameron]
1,285,Pirates of the Caribbean: At World's End,"[Captain, Barbossa,, long, believed, to, be, d...","[Adventure, Fantasy, Action]","[ocean, drugabuse, exoticisland, eastindiatrad...","[JohnnyDepp, OrlandoBloom, KeiraKnightley]",[GoreVerbinski]


In [37]:
# create a new column tags
movies['tags'] = movies['overview'] + movies['genres'] + movies['keywords'] + movies['cast'] + movies['director']
movies.head(2)

,movie_id,title,overview,genres,keywords,cast,director,tags
0,19995,Avatar,"[In, the, 22nd, century,, a, paraplegic, Marin...","[Action, Adventure, Fantasy, ScienceFiction]","[cultureclash, future, spacewar, spacecolony, ...","[SamWorthington, ZoeSaldana, SigourneyWeaver]",[JamesCameron],"[In, the, 22nd, century,, a, paraplegic, Marin..."
1,285,Pirates of the Caribbean: At World's End,"[Captain, Barbossa,, long, believed, to, be, d...","[Adventure, Fantasy, Action]","[ocean, drugabuse, exoticisland, eastindiatrad...","[JohnnyDepp, OrlandoBloom, KeiraKnightley]",[GoreVerbinski],"[Captain, Barbossa,, long, believed, to, be, d..."


In [38]:
new_df = movies[['movie_id', 'title', 'tags']]
new_df.head()

,movie_id,title,tags
0,19995,Avatar,"[In, the, 22nd, century,, a, paraplegic, Marin..."
1,285,Pirates of the Caribbean: At World's End,"[Captain, Barbossa,, long, believed, to, be, d..."
2,206647,Spectre,"[A, cryptic, message, from, Bond’s, past, send..."
3,49026,The Dark Knight Rises,"[Following, the, death, of, District, Attorney..."
4,49529,John Carter,"[John, Carter, is, a, war-weary,, former, mili..."


In [39]:
new_df['tags'] = new_df['tags'].apply(lambda x:" ".join(x))

<ipython-input-39-ad44e9ca1347>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['tags'] = new_df['tags'].apply(lambda x:" ".join(x))


In [40]:
new_df.head()

,movie_id,title,tags
0,19995,Avatar,"In the 22nd century, a paraplegic Marine is di..."
1,285,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha..."
2,206647,Spectre,A cryptic message from Bond’s past sends him o...
3,49026,The Dark Knight Rises,Following the death of District Attorney Harve...
4,49529,John Carter,"John Carter is a war-weary, former military ca..."


In [41]:
# change every word to lowercase
new_df['tags'] = new_df['tags'].apply(lambda x:x.lower())
new_df.head()

<ipython-input-41-217265fdb6f8>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['tags'] = new_df['tags'].apply(lambda x:x.lower())


,movie_id,title,tags
0,19995,Avatar,"in the 22nd century, a paraplegic marine is di..."
1,285,Pirates of the Caribbean: At World's End,"captain barbossa, long believed to be dead, ha..."
2,206647,Spectre,a cryptic message from bond’s past sends him o...
3,49026,The Dark Knight Rises,following the death of district attorney harve...
4,49529,John Carter,"john carter is a war-weary, former military ca..."


In [42]:
!pip install nltk

In [43]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [44]:
# stop word removal and stemming
from nltk.stem.porter import PorterStemmer
from nltk.corpus import stopwords
ps = PorterStemmer()
stopword_list = stopwords.words(fileids = 'english')


In [45]:
def stem(text):
    word_list = text.split()
    stemmed_text = " ".join([ps.stem(word) for word in word_list if word not in stopword_list])
    return stemmed_text

In [46]:
new_df['tags'] = new_df['tags'].apply(stem)

<ipython-input-46-be18a4346d89>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['tags'] = new_df['tags'].apply(stem)


In [47]:
new_df.head()

,movie_id,title,tags
0,19995,Avatar,"22nd century, parapleg marin dispatch moon pan..."
1,285,Pirates of the Caribbean: At World's End,"captain barbossa, long believ dead, come back ..."
2,206647,Spectre,cryptic messag bond’ past send trail uncov sin...
3,49026,The Dark Knight Rises,"follow death district attorney harvey dent, ba..."
4,49529,John Carter,"john carter war-weary, former militari captain..."


In [48]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features=10000)
vectors = cv.fit_transform(new_df['tags']).toarray()
vectors

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [49]:
list(cv.get_feature_names_out())

['00',
 '000',
 '007',
 '10',
 '11',
 '119',
 '11th',
 '12',
 '1200',
 '12th',
 '13',
 '14',
 '14th',
 '15',
 '150',
 '15th',
 '15thcenturi',
 '16',
 '16th',
 '16thcenturi',
 '17',
 '17th',
 '17thcenturi',
 '18',
 '1863',
 '18th',
 '18thcenturi',
 '19',
 '1910',
 '1912',
 '1920',
 '1930',
 '1930s',
 '1937',
 '1940',
 '1941',
 '1945',
 '1947',
 '1949',
 '1950',
 '1950s',
 '1952',
 '1959',
 '1960',
 '1965',
 '1970',
 '1971',
 '1972',
 '1976',
 '1979',
 '1980',
 '1980s',
 '1984',
 '1985',
 '1986',
 '1987',
 '1990',
 '1991',
 '1995',
 '1999',
 '19th',
 '19thcenturi',
 '20',
 '200',
 '2000',
 '2001',
 '2003',
 '2004',
 '2005',
 '2013',
 '2019',
 '20th',
 '21',
 '21st',
 '21stcenturi',
 '22nd',
 '23',
 '24',
 '25',
 '250',
 '30',
 '300',
 '3000',
 '3d',
 '40',
 '400',
 '47',
 '48',
 '50',
 '500',
 '50cent',
 '51',
 '60',
 '600',
 '60s',
 '66',
 '70',
 '747',
 '88',
 'aaron',
 'aaroneckhart',
 'aarontaylor',
 'aba',
 'abandon',
 'abandonedmin',
 'abbiecornish',
 'abduct',
 'abducted',
 'abiga

In [50]:
vectors.shape

(1831, 10000)

In [51]:
# calculate cosine distance between vectors -- each row with every other row
from sklearn.metrics.pairwise import cosine_similarity
similarity = cosine_similarity(vectors)
similarity

array([[1.        , 0.06933752, 0.07118685, ..., 0.07332356, 0.10050378,
        0.        ],
       [0.06933752, 1.        , 0.07897472, ..., 0.04067259, 0.05574947,
        0.        ],
       [0.07118685, 0.07897472, 1.        , ..., 0.02087869, 0.08585457,
        0.02177518],
       ...,
       [0.07332356, 0.04067259, 0.02087869, ..., 1.        , 0.08843154,
        0.03364316],
       [0.10050378, 0.05574947, 0.08585457, ..., 0.08843154, 1.        ,
        0.        ],
       [0.        , 0.        , 0.02177518, ..., 0.03364316, 0.        ,
        1.        ]])

In [52]:
similarity.shape

(1831, 1831)

In [53]:
similarity[0]

array([1.        , 0.06933752, 0.07118685, ..., 0.07332356, 0.10050378,
       0.        ])

In [54]:
list(enumerate(similarity[0]))

[(0, 1.0000000000000002),
 (1, 0.06933752452815364),
 (2, 0.07118684968143743),
 (3, 0.05986843400892498),
 (4, 0.15569978883230462),
 (5, 0.09021097956087903),
 (6, 0.030949223029508647),
 (7, 0.11785113019775792),
 (8, 0.048795003647426664),
 (9, 0.07647191129018727),
 (10, 0.08164965809277261),
 (11, 0.08006407690254358),
 (12, 0.07216878364870322),
 (13, 0.03247849012308155),
 (14, 0.11135885079684352),
 (15, 0.05),
 (16, 0.06454972243679029),
 (17, 0.11367471543078542),
 (18, 0.08018753738744804),
 (19, 0.06579516949597691),
 (20, 0.05986843400892498),
 (21, 0.09245003270420486),
 (22, 0.049507377148833714),
 (23, 0.07319250547113999),
 (24, 0.04508348173337162),
 (25, 0.038885688552390954),
 (26, 0.1212678125181665),
 (27, 0.14803913136594804),
 (28, 0.08333333333333336),
 (29, 0.050680303379961454),
 (30, 0.05786375623578448),
 (31, 0.1186447494690624),
 (32, 0.06950480468569159),
 (33, 0.07537783614444092),
 (34, 0.0),
 (35, 0.07537783614444092),
 (36, 0.14011213457945126),
 (3

In [55]:
sorted(list(enumerate(similarity[0])) ,reverse=True, key=lambda x:x[1])

[(0, 1.0000000000000002),
 (1214, 0.22213082915965968),
 (539, 0.21073180760588117),
 (507, 0.2027212135198458),
 (778, 0.2021130208636108),
 (1202, 0.20016019225635895),
 (582, 0.20003306604965967),
 (61, 0.19094065395649334),
 (1192, 0.18446619684315546),
 (942, 0.1803339269334865),
 (495, 0.17677669529663692),
 (322, 0.17407765595569782),
 (74, 0.16912035630063654),
 (972, 0.16839382851364088),
 (260, 0.16839382851364085),
 (1087, 0.16829726687457877),
 (47, 0.16448792373994225),
 (151, 0.1636634176769943),
 (577, 0.15779218606680068),
 (305, 0.1563858105428061),
 (4, 0.15569978883230462),
 (94, 0.15554275420956382),
 (1327, 0.15554275420956382),
 (172, 0.15309310892394862),
 (1199, 0.15075567228888184),
 (1342, 0.15061601901917734),
 (1069, 0.148969892882781),
 (27, 0.14803913136594804),
 (1440, 0.14731391274719743),
 (220, 0.14638501094227999),
 (1273, 0.14638501094227999),
 (83, 0.14547859349066158),
 (1827, 0.14288690166235204),
 (931, 0.14237369936287486),
 (661, 0.142133810903

In [56]:
def recommend(movie):
  #convert in lower case
  movie = movie.lower()

  # Check if the movie exists in new_df (case-insensitive)
  if movie not in new_df['title'].str.lower().values:
      print(f"The movie '{movie}' is not in the database.")
      return

  # Get the index of the movie in new_df
  movie_index = new_df[new_df['title'].str.lower() == movie].index[0]

  # Calculate movie similarities and recommend similar movies
  distances = similarity[movie_index]
  similar_movies = sorted(enumerate(distances), reverse=True, key=lambda x: x[1])[1:6]

  print(f"Recommendations for '{movie}':")
  for i in similar_movies:
      recommended_movie = new_df.iloc[i[0]].title
      print(recommended_movie)



In [57]:
recommend('Batman')

Recommendations for 'batman':
Batman & Robin
The Dark Knight Rises
Batman Returns
The Dark Knight
Batman Begins


In [58]:
recommend('aliens')

The movie 'aliens' is not in the database.


In [59]:
recommend('the matrix')

Recommendations for 'the matrix':
The Matrix Revolutions
The Matrix Reloaded
Hackers
Jupiter Ascending
Æon Flux


In [60]:
import pickle

In [61]:
pickle.dump(new_df, open('movies.pkl', 'wb'))

In [62]:
new_df['title'].values

array(['Avatar', "Pirates of the Caribbean: At World's End", 'Spectre',
       ..., 'Cloverfield',
       'Teenage Mutant Ninja Turtles II: The Secret of the Ooze',
       'The Untouchables'], dtype=object)

In [63]:
new_df.to_dict()

{'movie_id': {0: 19995,
  1: 285,
  2: 206647,
  3: 49026,
  4: 49529,
  5: 559,
  6: 38757,
  7: 99861,
  8: 767,
  9: 209112,
  10: 1452,
  11: 10764,
  12: 58,
  13: 57201,
  14: 49521,
  15: 2454,
  16: 24428,
  17: 1865,
  18: 41154,
  19: 122917,
  20: 1930,
  21: 20662,
  22: 57158,
  23: 2268,
  24: 254,
  25: 597,
  26: 271110,
  27: 44833,
  28: 135397,
  29: 37724,
  30: 558,
  31: 68721,
  32: 12155,
  33: 36668,
  34: 62211,
  35: 8373,
  36: 91314,
  37: 68728,
  38: 102382,
  39: 20526,
  40: 49013,
  41: 44912,
  42: 10193,
  43: 534,
  44: 168259,
  45: 72190,
  46: 127585,
  47: 54138,
  48: 81005,
  49: 64682,
  50: 9543,
  51: 68726,
  52: 38356,
  53: 217,
  54: 105864,
  55: 62177,
  56: 188927,
  57: 10681,
  58: 5174,
  59: 14161,
  60: 17979,
  61: 76757,
  62: 258489,
  63: 411,
  64: 246655,
  65: 155,
  66: 14160,
  67: 15512,
  68: 1726,
  69: 44826,
  70: 8487,
  71: 1735,
  72: 297761,
  73: 2698,
  74: 137113,
  75: 9804,
  76: 14869,
  77: 150540,
  78:

In [64]:
pickle.dump(new_df.to_dict(),open('movies_dict.pkl','wb'))

In [65]:
pickle.dump(similarity,open('similarity.pkl','wb'))